In [23]:
# import essential libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import os
import random
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [39]:
# specify the way to load the data

train_path = 'dataset/train'
test_path = 'dataset/test'
valid_path = 'dataset/valid'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
.flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
.flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
.flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [3]:
# normal model architecture
'''
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
'''

"\nmodel = Sequential()\nmodel.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)))\nmodel.add(MaxPool2D(pool_size=(2,2), strides=2))\nmodel.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))\nmodel.add(MaxPool2D(pool_size=(2,2), strides=2))\nmodel.add(Flatten())\nmodel.add(Dense(2,activation='softmax'))\n"

In [40]:
# we will use the Mobilenet model for our problem for better accuracy

# download the model
mobilenet = keras.applications.mobilenet.MobileNet()

In [41]:
mobilenet.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [42]:
# we will build a sequential model and then add all layers of vgg16_model in that model

model = Sequential()

for layer in mobilenet.layers[:-1]:
    model.add(layer)

In [43]:
# summary of our new model

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)     

In [44]:
# Now freeze all layers of this model

for layer in model.layers:
    layer.trainable = False

In [48]:
# add new dense layer for the as output layer

model.add(Dense(units=2, activation='softmax'))

In [49]:
# specify the loss, optimizer and metrics parameters

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# now fit the model
model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=1)

Epoch 1/5
100/100 [==============================] - 119s 1s/step - loss: 1.0862 - accuracy: 0.5000 - val_loss: 1.0065 - val_accuracy: 0.5000
Epoch 2/5
100/100 [==============================] - 104s 1s/step - loss: 0.9526 - accuracy: 0.5000 - val_loss: 0.9015 - val_accuracy: 0.5000
Epoch 3/5
100/100 [==============================] - 108s 1s/step - loss: 0.8602 - accuracy: 0.5000 - val_loss: 0.8213 - val_accuracy: 0.5000
Epoch 4/5
100/100 [==============================] - 101s 1s/step - loss: 0.7931 - accuracy: 0.5000 - val_loss: 0.7675 - val_accuracy: 0.5000
Epoch 5/5
100/100 [==============================] - 103s 1s/step - loss: 0.7490 - accuracy: 0.5000 - val_loss: 0.7333 - val_accuracy: 0.5000
